# ⚡ MIR GPU ULTRA-TURBO: TorchAudio & nnAudio
### *"Lightning Speed Audio Analysis Dashboard"*

Has pasado de la lentitud de la CPU (Librosa) a la velocidad pura de la GPU. Este cuaderno utiliza **TorchAudio** para carga de alta velocidad y **nnAudio** para transformadas matemáticas paralelas.

| Motor | Velocidad Estimada | Tecnología |
| :--- | :--- | :--- |
| **Librosa (CPU)** | ~3.0s / pista | Single-core FFT |
| **Turbo GPU** | **~0.05s / pista** | CUDA Matrix Ops |

---

In [ ]:
!pip install -q nnAudio torchaudio torch pandas matplotlib tqdm

import os
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from nnAudio import features
from IPython.display import Audio, display, FileLink
from tqdm.auto import tqdm

# Prevenir errores de visualización de Librosa (usaremos solo para specview)
import librosa
import librosa.display

# Configuración de Dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 TURBO ACTIVADO: {device.upper()}")

# Estética de Alto Impacto
plt.style.use('dark_background')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (16, 8), 'axes.grid': False})

OUTPUT_DIR = 'ultimate_gpu_results'
os.makedirs(os.path.join(OUTPUT_DIR, 'plots'), exist_ok=True)

## 📂 1. Dataset Distributivo
Configuración de rutas de Kaggle (Slavic Songs).

In [ ]:
DATASET_PATH = '/kaggle/input/datasets/danieldobles/slavic-songs'
IGNORE = 'REF.flac'

if not os.path.exists(DATASET_PATH):
    DATASET_PATH = 'Slavic Data_Set'

files = [f for f in os.listdir(DATASET_PATH) if f.endswith(('.mp3', '.wav', '.flac')) and f != IGNORE]
print(f"💿 Archivos detectados: {len(files)}")

## ⚡ 2. Inicialización de Kernels CUDA
Inicializamos los motores de transformación rítmica y tonal en la VRAM.

In [ ]:
SAMPLE_RATE = 44100

# Capas de nnAudio (Optimizadas para CUDA)
stft_gpu = features.STFT(n_fft=2048, hop_length=512).to(device)
mel_gpu = features.MelSpectrogram(sr=SAMPLE_RATE, n_fft=2048, n_mels=128).to(device)
cqt_gpu = features.CQT2010v2(sr=SAMPLE_RATE, hop_length=512, fmin=32.7, n_bins=84).to(device)

print("✨ Motores GPU listos para la carnicería de datos.")

## 🧬 3. El Buffer de Procesamiento "Forensic"
Carga con TorchAudio (I/O Paralelo) y procesamiento nnAudio.

In [ ]:
def forensic_analysis(file_path):
    fname = os.path.basename(file_path)
    
    # 1. Carga Ultra-Rápida con TorchAudio
    waveform, sr = torchaudio.load(file_path)
    
    # Resample si es necesario para los kernels fijos
    if sr != SAMPLE_RATE:
        resampler = torchaudio.transforms.Resample(sr, SAMPLE_RATE).to(waveform.device)
        waveform = resampler(waveform)
    
    # Aseguramos mono y movemos a GPU
    waveform = torch.mean(waveform, dim=0, keepdim=True).to(device)
    
    start_time = time.time()
    
    # 2. Inferencia Espectral GPU
    with torch.no_grad():
        melspec = mel_gpu(waveform)
        cqt_map = cqt_gpu(waveform)
        
        # Transformar a dB para visualización
        mel_db = torchaudio.transforms.AmplitudeToDB()(melspec).cpu().squeeze().numpy()
        cqt_db = torchaudio.transforms.AmplitudeToDB()(torch.abs(cqt_map)).cpu().squeeze().numpy()
        
    gpu_time = time.time() - start_time
    
    # 3. Métricas Complementarias (Solo 1 vez por muestra)
    y_np = waveform.cpu().squeeze().numpy()
    tempo, _ = librosa.beat.beat_track(y=y_np, sr=SAMPLE_RATE)
    
    # Guardar Gráfico 'Brutal'
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 1, 1)
    librosa.display.specshow(mel_db, x_axis='time', y_axis='mel', sr=SAMPLE_RATE, cmap='magma')
    plt.title(f"FORENSIC MEL: {fname} | GPU Task: {gpu_time:.4f}s")
    plt.colorbar(format='%+2.0f dB')
    
    plt.subplot(2, 1, 2)
    librosa.display.specshow(cqt_db, x_axis='time', y_axis='cqt_note', sr=SAMPLE_RATE, cmap='inferno')
    plt.title(f"CQT ANALYSIS: {tempo:.1f} BPM")
    plt.colorbar(format='%+2.0f dB')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'plots', f"{fname}_ultra.png"))
    plt.close()
    
    return {"file": fname, "bpm": tempo, "gpu_time": gpu_time}

## 🚀 4. Lanzamiento de la Carnicería Turbo

In [ ]:
import time
results = []

print("⚡ Iniciando secuencia Turbo Mode...")
for f in tqdm(files, desc="Análisis a la velocidad del rayo"):
    try:
        res = forensic_analysis(os.path.join(DATASET_PATH, f))
        results.append(res)
    except Exception as e:
        print(f"❌ ERROR en {f}: {e}")

df = pd.DataFrame(results)
df.to_csv(os.path.join(OUTPUT_DIR, 'turbo_report.csv'), index=False)
display(df.head())

print(f"\n🥇 Velocidad Media GPU: {df['gpu_time'].mean():.5f} segundos por análisis espectral.")

## 📦 5. Exportación Automática

In [ ]:
ZIP_NAME = 'MIR_TURBO_GPU_RESULTS.zip'
shutil.make_archive('MIR_TURBO_GPU_RESULTS', 'zip', OUTPUT_DIR)

print(f"\n🔥 Archivo empaquetado: {ZIP_NAME}")
display(FileLink(ZIP_NAME, result_html_prefix="🚀 DESCARGA EL BOTÍN AQUÍ: "))